### Fine tuning SAM with OMERO data using a batch approach

TO DO
- make it work for different OMERO data types (single images, plates, etc)
- clean up tmp files when not neccesary anymore
- store all annotations into OMERO, see: https://github.com/computational-cell-analytics/micro-sam/issues/445; in series annotator possible to add commit path with prompts, but they get overwritten
- clean up the errors and warnings output from napari

Instructions:
  - To make it easier to run with OMERO and to not expose login and passwords password is stored in .env file (see example .env_example) . Still it is not recommended to save credentials unencrypted hence a better solution will be worked on.

In [ ]:
# OMERO-related imports
import omero
from omero.gateway import BlitzGateway
import ezomero

# Scientific computing and image processing
import cv2
import imageio.v3 as imageio
import numpy as np
import pandas as pd

# File and system operations
import os
import shutil
import tempfile
import zipfile
import warnings
from tifffile import imwrite
from dotenv import load_dotenv

# Dask and Zarr for lazy loading and processing
import dask
import dask.array as da
import zarr

# Micro-SAM and Napari
from napari.settings import get_settings
from micro_sam.sam_annotator import image_series_annotator, annotator_2d
from micro_sam.util import precompute_image_embeddings
import napari

### Setup connection with OMERO

In [ ]:
load_dotenv(override=True)
conn = BlitzGateway(host=os.environ.get("HOST"), username=os.environ.get("USER_NAME"), passwd=os.environ.get("PASSWORD"), secure=True)
connection_status = conn.connect()
if connection_status:
    print("Connected to OMERO Server")
else:
    print("Connection to OMERO Server Failed")
conn.c.enableKeepAlive(60)

omero.services.universiteitleiden.nl


INFO:omero.gateway:created connection (uuid=69d53fac-cfbb-46a1-9350-a9a08dafcefe)
INFO:omero.util.Resources:Starting


Connected to OMERO Server


### Get info from the dataset

In [9]:
datatype = "dataset" # "plate", "dataset", "image"
data_id = 24143
nucl_channel = 0

#validate that data_id matches datatype
if datatype == "plate":
    plate = conn.getObject("Plate", data_id)
    print('Plate Name: ', plate.getName())
elif datatype == "dataset":
    dataset = conn.getObject("Dataset", data_id)
    print('Dataset Name: ', dataset.getName())
elif datatype == "image":
    image = conn.getObject("Image", data_id)
    print('Image Name: ', image.getName())

Dataset Name:  tmp


### Create temporary folder to store training data, this will be uploaded to OMERO later

In [10]:
output_directory = os.path.normcase(tempfile.mkdtemp())
print('Output Directory: ', output_directory)

Output Directory:  c:\users\maarten\appdata\local\temp\tmp7o1lxv8h


In [17]:

def zip_directory(source_path, zarr_path, zip_file):
    """Zip a directory while handling null characters in paths."""
    for root, dirs, files in os.walk(zarr_path):
        for file in files:
            try:
                # Create paths
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, source_path)
                
                # Remove null characters while preserving the path structure
                safe_full_path = full_path.replace('\x00', '')
                safe_rel_path = rel_path.replace('\x00', '')
                
                # Add file to zip if it exists
                if os.path.exists(safe_full_path):
                    zip_file.write(safe_full_path, safe_rel_path)
            except Exception as e:
                print(f"Warning: Error processing {file}: {str(e)}")
                continue

def interleave_arrays(train_images, validate_images):
    """
    Interleave two arrays of images in the pattern: train[0], validate[0], train[1], validate[1], ...
    If arrays are of unequal length, remaining elements are appended at the end.
    """
    # Create empty list to store interleaved images
    interleaved = []
    sequence = []
    # Get the length of the longer array
    max_len = max(len(train_images), len(validate_images))
    
    # Interleave the arrays
    for i in range(max_len):
        # Add train image if available
        if i < len(train_images):
            interleaved.append(train_images[i])
            sequence.append(0)
        # Add validate image if available
        if i < len(validate_images):
            interleaved.append(validate_images[i])
            sequence.append(1)
    
    return np.array(interleaved), np.array(sequence)


def mask_to_contour(mask):
    """Converts a binary mask to a list of ROI coordinates.

    Args:
        mask (np.ndarray): binary mask

    Returns:
        list: list of ROI coordinates
    """
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def label_to_rois(label_img, z_slice, channel, timepoint):
    """
    Convert a 2D label image to OMERO ROI shapes
    
    Args:
        label_img (np.ndarray): 2D labeled image
        z_slice (int): Z-slice index
        channel (int): Channel index
        timepoint (int): Time point index
    
    Returns:
        list: List of OMERO shape objects
    """
    shapes = []
    unique_labels = np.unique(label_img)
    
    # Skip background (label 0)
    for label in unique_labels[1:]:
        # Create binary mask for this label
        mask = (label_img == label).astype(np.uint8)
        
        # Get contours
        contours = mask_to_contour(mask)
        
        # Convert each contour to polygon ROI
        for contour in contours:
            contour = contour[:, 0, :]  # Reshape to (N, 2)
            # Create polygon without text parameter
            poly = ezomero.rois.Polygon(
                points=contour,  # explicitly name the points parameter
                z=z_slice,
                c=channel,
                t=timepoint
            )
            shapes.append(poly)
    
    return shapes

def upload_rois_and_labels(conn, image, label_file, z_slice, channel, timepoint, model_type):
    """
    Upload both label map and ROIs for a segmented image
    
    Args:
        conn: OMERO connection
        image: OMERO image object
        label_file: Path to the label image file
        z_slice: Z-slice index
        channel: Channel index
        timepoint: Time point index
        model_type: SAM model type used
    
    Returns:
        tuple: (label_id, roi_id)
    """
    # Upload label map as attachment
    label_id = ezomero.post_file_annotation(
        conn,
        str(label_file),
        ns='microsam.labelimage',
        object_type="Image",
        object_id=image.getId(),
        description=f'SAM segmentation ({model_type})'
    )
    
    # Create ROIs from label image
    label_img = imageio.imread(label_file)
    shapes = label_to_rois(label_img, z_slice, channel, timepoint)
    
    if shapes:  # Only create ROI if shapes were found
        roi_id = ezomero.post_roi(
            conn,
            image.getId(),
            shapes,
            name=f'SAM_{model_type}',
            description=f'Segmentation using SAM model {model_type}'
        )
    else:
        roi_id = None
        
    return label_id, roi_id

def process_omero_batch(
    dataset,
    output_folder: str,
    model_type: str = 'vit_l',
    batch_size: int = 3,
    channel: int = 0,
    timepoint: int = 0,
    z_slice: int = 0,
    segment_all: bool = True,
    train_n: int = 3,
    validate_n: int = 3,
):

    """
    Process OMERO dataset in batches for SAM segmentation and upload results back to OMERO
    
    Args:
        dataset: OMERO dataset object
        output_folder: Path to store temporary files
        model_type: SAM model type
        batch_size: Number of images to process at once
        channel: Channel to segment
        timepoint: Timepoint to process
        z_slice: Z-slice to process
        segment_all: segment all images in the dataset or only 
        
    """
    # Setup output directories
    output_path = os.path.join(output_folder, "output")
    embed_path = os.path.join(output_folder, "embed")
    
    # Remove directories if they exist
    if os.path.exists(output_path):
        shutil.rmtree(output_path)
    if os.path.exists(embed_path):
        shutil.rmtree(embed_path)
        
    # Create fresh directories
    os.makedirs(output_path)
    os.makedirs(embed_path)
    images_dataset = list(dataset.listChildren())
    if segment_all:
    # Get all images from dataset
        combine_images = images_dataset
    else:
        #pick 5 random images to train on, check first if there are enough images for training an validation
        if len(images_dataset) < train_n + validate_n:
            print("Not enough images in dataset for training and validation")
            assert False
        else:
            #pick images for training and validation
            train_images = np.random.choice(images_dataset, train_n, replace=False)
            validate_images = np.random.choice([x for x in images_dataset if x not in train_images], validate_n, replace=False)
            combined_images,combine_images_sequence = interleave_arrays(train_images, validate_images)
    total_batches = (len(combined_images) + batch_size - 1) // batch_size
    df = pd.DataFrame(columns=[
        "image_id", "image_name", "train", "validate", 
        "channel", "timepoint", "sam_model", "embed_id", "label_id", "roi_id"
    ])
    
    # Process images in batches
    for batch_idx in range(total_batches):
    
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(combined_images))
        batch_images = combined_images[start_idx:end_idx]
        
        # Load batch images
        images = []
        for image in batch_images:
            pixels = image.getPrimaryPixels()
            img = pixels.getPlane(z_slice, channel, timepoint)
            images.append(img)
        
        # Process batch with SAM
        viewer = napari.Viewer()
        image_series_annotator(
            images, 
            model_type=model_type,
            viewer=viewer,
            embedding_path=os.path.join(output_folder, "embed"),
            output_folder=os.path.join(output_folder, "output")
        )
        
        napari.run()
        print("Done annotating batch, uploading to OMERO now")
        # Upload results for batch
        for n, image in enumerate(batch_images):
            local_n = n
            global_n = start_idx + n
            
            # Upload embedding
            embed_zarr = f"embedding_{local_n:05d}.zarr"
            embed_path = os.path.join(output_folder, "embed")
            zip_path = os.path.join(output_folder, "embed", f"embedding_{local_n:05d}.zip")
            
            with zipfile.ZipFile(zip_path, 'w') as zip_file:
                zip_directory(embed_path, embed_zarr, zip_file)
            
            embed_id = ezomero.post_file_annotation(
                conn,
                str(zip_path),
                ns='microsam.embeddings',
                object_type="Image",
                object_id=image.getId(),
                description=f'SAM embedding ({model_type})'
            )
            
            # Upload labels and ROIs
            label_file = os.path.join(output_folder, "output", f"seg_{local_n:05d}.tif")
            label_id, roi_id = upload_rois_and_labels(
                conn, 
                image, 
                label_file, 
                z_slice, 
                channel, 
                timepoint, 
                model_type
            )
            
            # Update tracking dataframe
            new_row = pd.DataFrame([{
                "image_id": image.getId(),
                "image_name": image.getName(),
                "train": global_n % 2 == 0,
                "validate": global_n % 2 == 1,
                "channel": channel,
                "z_slice": z_slice,
                "timepoint": timepoint,
                "sam_model": model_type,
                "embed_id": embed_id,
                "label_id": label_id,
                "roi_id": roi_id
            }])
            df = pd.concat([df, new_row], ignore_index=True)
        
        # Clean up batch files
        for n in range(batch_size):  # Use local indexing for cleanup
            embed_zip = os.path.join(output_folder, "embed", f"embedding_{n:05d}.zip")
            embed_zarr = os.path.join(output_folder, "embed", f"embedding_{n:05d}.zarr")
            seg_file = os.path.join(output_folder, "output", f"seg_{n:05d}.tif")
            
            if os.path.exists(embed_zip):
                os.remove(embed_zip)
            if os.path.exists(embed_zarr):
                shutil.rmtree(embed_zarr)
            if os.path.exists(seg_file):
                os.remove(seg_file)
    
    # Upload final tracking table
    table_id = ezomero.post_table(
        conn, 
        object_type="Dataset", 
        object_id=dataset.getId(), 
        table=df,
        title="micro_sam_training_data"
    )
    
    return table_id, combined_images


### Load images from OMERO and open in napari with micro-sam annotator

### Running segmentation in batch

Note: some warnings from napari are expected in the output here, generally not a problem

In [19]:
##imput parameters
model_type = 'vit_b'
segment_all = False
train_n = 2   
validate_n = 2
channel = 3 #which channel to segment starting from 0
timepoint = 0
z_slice = 4 #for now pick one slice but TODO add option to pick multiple slices by giving a list of z slices, or random slices
batch_size = 2 # the number of images to process at once in napari

#set napari settings
settings = get_settings()
settings.application.ipy_interactive = False

# Usage
if datatype == "dataset":
    settings = get_settings()
    settings.application.ipy_interactive = False
    
    table_id,combined_images = process_omero_batch(
        dataset=dataset,
        output_folder=output_directory,
        model_type=model_type,
        batch_size=batch_size,
        channel=channel,
        timepoint=timepoint,
        z_slice=z_slice,
        segment_all=segment_all,
        train_n=train_n,
        validate_n=validate_n,
    )
    print("Table ID:", table_id)

INFO:omero.gateway:Registered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/98cdc1da-c116-495a-9b21-172e2f3d4582omero.api.RawPixelsStore -t -e 1.1:tcp -h 10.10.100.172 -p 40304 -t 60000:tcp -h 132.229.92.78 -p 40304 -t 60000
INFO:omero.gateway:Unregistered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/98cdc1da-c116-495a-9b21-172e2f3d4582omero.api.RawPixelsStore -t -e 1.1:tcp -h 10.10.100.172 -p 40304 -t 60000:tcp -h 132.229.92.78 -p 40304 -t 60000
INFO:omero.gateway:Registered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/405204b0-2813-453a-b28c-65de6281143domero.api.RawPixelsStore -t -e 1.1:tcp -h 10.10.100.172 -p 40304 -t 60000:tcp -h 132.229.92.78 -p 40304 -t 60000
INFO:omero.gateway:Unregistered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/405204b0-2813-453a-b28c-65de6281143domero.api.RawPixelsStore -t -e 1.1:tcp -h 10.10.100.172 -p 40304 -t 60000:tcp -h 132.229.92.78 -p 40304 -t 60000


Precomputation took 15.528432369232178 seconds (= 00:16 minutes)
The first image to annotate is image number 0
Loading next image: at index 1
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
File c:\Users\Maarten\miniforge3\envs\micro-sam\Lib\site-packages\psygnal\_signal.py:1196, in SignalInstance._run_emit_loop(self=<SignalInstance 'changed' on PushButton(value=False, annotation=None, name='call_button')>, args=(False,))
   1195     with Signal._emitting(self):
-> 1196         self._run_emit_loop_inner()
        self = <SignalInstance 'changed' on PushButton(value=False, annotation=None, name='call_button')>
        self._run_emit_loop_inner = <bound method SignalInstance._run_emit_loop_immediate of <SignalInstance 'changed' on PushButton(value=False, annotation=None, name='call_button')>>
   1197 except RecursionError as e:

File c:\Users\Maarten\miniforge3\envs\micro-sam\Lib\site-

C:\Users\Maarten\AppData\Local\Temp\ipykernel_924\3220352429.py:223: UserWarning: Refusing to run a QApplication with no topLevelWidgets. To run the app anyway, use `run(force=True)`
  napari.run()


Done annotating batch, uploading to OMERO now


INFO:omero.gateway:Registered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/eddecad4-4f6f-44c9-bc52-85a6e508b099omero.api.RawFileStore -t -e 1.1:tcp -h 10.10.100.172 -p 40304 -t 60000:tcp -h 132.229.92.78 -p 40304 -t 60000
INFO:omero.gateway:Unregistered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/eddecad4-4f6f-44c9-bc52-85a6e508b099omero.api.RawFileStore -t -e 1.1:tcp -h 10.10.100.172 -p 40304 -t 60000:tcp -h 132.229.92.78 -p 40304 -t 60000
INFO:omero.gateway:Registered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/b92235e8-52ee-4030-963d-9ce8174d2717omero.api.RawFileStore -t -e 1.1:tcp -h 10.10.100.172 -p 40304 -t 60000:tcp -h 132.229.92.78 -p 40304 -t 60000
INFO:omero.gateway:Unregistered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/b92235e8-52ee-4030-963d-9ce8174d2717omero.api.RawFileStore -t -e 1.1:tcp -h 10.10.100.172 -p 40304 -t 60000:tcp -h 132.229.92.78 -p 40304 -t 60000
INFO:omero.gateway:Registered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/150042e3-bb8d-403c-a44e-21d5159de8dfomero.api.RawFileStore -t -e 1.1:tcp 

Precomputation took 15.948132038116455 seconds (= 00:16 minutes)
The first image to annotate is image number 0
Loading next image: at index 1
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
File c:\Users\Maarten\miniforge3\envs\micro-sam\Lib\site-packages\psygnal\_signal.py:1196, in SignalInstance._run_emit_loop(self=<SignalInstance 'changed' on PushButton(value=False, annotation=None, name='call_button')>, args=(False,))
   1195     with Signal._emitting(self):
-> 1196         self._run_emit_loop_inner()
        self = <SignalInstance 'changed' on PushButton(value=False, annotation=None, name='call_button')>
        self._run_emit_loop_inner = <bound method SignalInstance._run_emit_loop_immediate of <SignalInstance 'changed' on PushButton(value=False, annotation=None, name='call_button')>>
   1197 except RecursionError as e:

File c:\Users\Maarten\miniforge3\envs\micro-sam\Lib\site-

C:\Users\Maarten\AppData\Local\Temp\ipykernel_924\3220352429.py:223: UserWarning: Refusing to run a QApplication with no topLevelWidgets. To run the app anyway, use `run(force=True)`
  napari.run()


Done annotating batch, uploading to OMERO now


INFO:omero.gateway:Registered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/bfc13585-e18b-4122-8226-74b45ae7baabomero.api.RawFileStore -t -e 1.1:tcp -h 10.10.100.172 -p 40304 -t 60000:tcp -h 132.229.92.78 -p 40304 -t 60000
INFO:omero.gateway:Unregistered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/bfc13585-e18b-4122-8226-74b45ae7baabomero.api.RawFileStore -t -e 1.1:tcp -h 10.10.100.172 -p 40304 -t 60000:tcp -h 132.229.92.78 -p 40304 -t 60000
INFO:omero.gateway:Registered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/46781d8a-b4cc-4ae6-98fd-e3e99f9f4fbeomero.api.RawFileStore -t -e 1.1:tcp -h 10.10.100.172 -p 40304 -t 60000:tcp -h 132.229.92.78 -p 40304 -t 60000
INFO:omero.gateway:Unregistered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/46781d8a-b4cc-4ae6-98fd-e3e99f9f4fbeomero.api.RawFileStore -t -e 1.1:tcp -h 10.10.100.172 -p 40304 -t 60000:tcp -h 132.229.92.78 -p 40304 -t 60000
INFO:omero.gateway:Registered 69d53fac-cfbb-46a1-9350-a9a08dafcefe/373a23eb-34b1-4d14-8dea-174520c7a24domero.api.RawFileStore -t -e 1.1:tcp 

Table ID: 124167


### Dask Lazy Loading Functions for OMERO Data

In [ ]:
def get_dask_image(conn, image_id, z_slice=None, timepoint=None, channel=None):
    """
    Get a dask array representation of an OMERO image for lazy loading
    
    Args:
        conn: OMERO connection
        image_id: ID of image to load
        z_slice: Optional specific Z slice to load (int or list)
        timepoint: Optional specific timepoint to load (int or list)
        channel: Optional specific channel to load (int or list)
    
    Returns:
        dask array representation of image
    """
    image = conn.getObject("Image", image_id)
    pixels = image.getPrimaryPixels()
    
    # Get image dimensions
    size_z = image.getSizeZ()
    size_c = image.getSizeC()
    size_t = image.getSizeT()
    size_y = image.getSizeY()
    size_x = image.getSizeX()
    
    # Define specific dimensions to load if provided
    z_range = [z_slice] if isinstance(z_slice, int) else (range(size_z) if z_slice is None else z_slice)
    t_range = [timepoint] if isinstance(timepoint, int) else (range(size_t) if timepoint is None else timepoint)
    c_range = [channel] if isinstance(channel, int) else (range(size_c) if channel is None else channel)
    
    # Create empty list to store delayed objects
    delayed_planes = {}
    
    print(f"Creating dask array for image {image_id} with lazy loading")
    print(f"Dimensions: Z={len(z_range)}, C={len(c_range)}, T={len(t_range)}, Y={size_y}, X={size_x}")
    
    # Create lazy loading function
    @dask.delayed
    def get_plane(z, c, t):
        print(f"Loading plane: Z={z}, C={c}, T={t}")
        return pixels.getPlane(z, c, t)
    
    # Build dask arrays
    arrays = []
    for t in t_range:
        t_arrays = []
        for z in z_range:
            z_arrays = []
            for c in c_range:
                # Create a key for this plane
                key = (z, c, t)
                
                # Check if we've already created this delayed object
                if key not in delayed_planes:
                    # Create a delayed object for this plane
                    delayed_plane = get_plane(z, c, t)
                    delayed_planes[key] = delayed_plane
                else:
                    delayed_plane = delayed_planes[key]
                
                # Convert to dask array with known shape and dtype
                shape = (size_y, size_x)
                dtype = np.uint16  # Most OMERO images use 16-bit
                dask_plane = da.from_delayed(delayed_plane, shape=shape, dtype=dtype)
                z_arrays.append(dask_plane)
            if z_arrays:
                # Stack channels for this z position
                t_arrays.append(da.stack(z_arrays))
        if t_arrays:
            # Stack z-planes for this timepoint
            arrays.append(da.stack(t_arrays))
    
    if arrays:
        # Stack all timepoints
        return da.stack(arrays)
    else:
        return None

def store_annotations_in_zarr(mask_data, output_folder, image_num):
    """
    Store annotation masks in zarr format for efficient access
    
    Args:
        mask_data: Numpy array with mask data
        output_folder: Base folder to store zarr data
        image_num: Image number/identifier
        
    Returns:
        path: Path to the zarr store
    """
    # Create zarr directory if it doesn't exist
    zarr_dir = os.path.join(output_folder, "annotations")
    os.makedirs(zarr_dir, exist_ok=True)
    
    # Create zarr filename
    zarr_path = os.path.join(zarr_dir, f"annotation_{image_num:05d}.zarr")
    
    # Remove existing zarr store if it exists
    if os.path.exists(zarr_path):
        shutil.rmtree(zarr_path)
        
    # Create zarr array from mask data
    z = zarr.open(zarr_path, mode='w')
    z.create_dataset('masks', data=mask_data, chunks=(256, 256))
    
    # Return path to zarr store
    return zarr_path

def zarr_to_tiff(zarr_path, output_tiff_path):
    """
    Convert zarr store to TIFF file for OMERO upload
    
    Args:
        zarr_path: Path to zarr store
        output_tiff_path: Path to save TIFF file
        
    Returns:
        output_tiff_path: Path to saved TIFF file
    """
    # Load data from zarr
    z = zarr.open(zarr_path, mode='r')
    mask_data = z['masks'][:]
    
    # Save as TIFF
    imwrite(output_tiff_path, mask_data)
    
    return output_tiff_path

def process_omero_batch_with_dask(
    dataset,
    output_folder: str,
    model_type: str = 'vit_l',
    batch_size: int = 3,
    channel: int = 0,
    timepoint: int = 0,
    z_slice: int = 0,
    segment_all: bool = True,
    train_n: int = 3,
    validate_n: int = 3,
):

    """
    Process OMERO dataset in batches for SAM segmentation using dask for lazy loading
    and zarr for temporary annotation storage
    
    Args:
        dataset: OMERO dataset object
        output_folder: Path to store temporary files
        model_type: SAM model type
        batch_size: Number of images to process at once
        channel: Channel to segment
        timepoint: Timepoint to process
        z_slice: Z-slice to process
        segment_all: segment all images in the dataset or only train/validate subset
        train_n: Number of training images if not segment_all
        validate_n: Number of validation images if not segment_all
    
    Returns:
        tuple: (table_id, combined_images)
    """
    # Setup output directories
    output_path = os.path.join(output_folder, "output")
    embed_path = os.path.join(output_folder, "embed")
    zarr_path = os.path.join(output_folder, "zarr")
    
    # Remove directories if they exist
    for path in [output_path, embed_path, zarr_path]:
        if os.path.exists(path):
            shutil.rmtree(path)
        os.makedirs(path)
        
    # Get images from dataset
    images_dataset = list(dataset.listChildren())
    
    # Select images based on segment_all flag
    if segment_all:
        combined_images = images_dataset
        combined_images_sequence = np.zeros(len(combined_images))  # All treated as training
    else:
        # Check if we have enough images
        if len(images_dataset) < train_n + validate_n:
            print("Not enough images in dataset for training and validation")
            raise ValueError(f"Need at least {train_n + validate_n} images but found {len(images_dataset)}")
            
        # Select random images for training and validation
        train_images = np.random.choice(images_dataset, train_n, replace=False)
        validate_images = np.random.choice([x for x in images_dataset if x not in train_images], validate_n, replace=False)
        combined_images, combined_images_sequence = interleave_arrays(train_images, validate_images)
    
    # Calculate total number of batches
    total_batches = (len(combined_images) + batch_size - 1) // batch_size
    df = pd.DataFrame(columns=[
        "image_id", "image_name", "train", "validate", 
        "channel", "z_slice", "timepoint", "sam_model", "embed_id", "label_id", "roi_id"
    ])
    
    print(f"Processing {len(combined_images)} images in {total_batches} batches")
    
    # Process images in batches
    for batch_idx in range(total_batches):
        print(f"\nProcessing batch {batch_idx+1}/{total_batches}")
        
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(combined_images))
        batch_images = combined_images[start_idx:end_idx]
        
        # Load batch images as dask arrays for lazy loading
        conn = dataset.getConnection()
        images = []
        dask_images = []
        image_ids = []
        
        for image in batch_images:
            image_ids.append(image.getId())
            
            # Create both standard numpy array and dask array
            # Standard array is needed for current micro-sam compatibility
            pixels = image.getPrimaryPixels()
            img = pixels.getPlane(z_slice, channel, timepoint)
            images.append(img)
            
            print(f"Creating dask array for image {image.getId()}")
            dask_img = get_dask_image(conn, image.getId(), z_slice=z_slice, 
                                    timepoint=timepoint, channel=channel)
            dask_images.append(dask_img)
        
        # Process batch with SAM using standard numpy arrays for now
        # Note: In the future, micro-sam could be updated to work directly with dask
        print("Starting napari viewer with SAM annotator")
        viewer = napari.Viewer()
        
        # Add image series annotator
        image_series_annotator(
            images, 
            model_type=model_type,
            viewer=viewer,
            embedding_path=os.path.join(output_folder, "embed"),
            output_folder=os.path.join(output_folder, "output")
        )
        
        napari.run()
        print("Done annotating batch, storing results in zarr and uploading to OMERO")
        
        # Process results for batch
        for n, image in enumerate(batch_images):
            local_n = n  # Index within current batch
            global_n = start_idx + n  # Global index across all batches
            
            # Store segmentation mask in zarr before uploading to OMERO
            seg_file_path = os.path.join(output_folder, "output", f"seg_{local_n:05d}.tif")
            if not os.path.exists(seg_file_path):
                print(f"Warning: Segmentation file not found for image {image.getId()}")
                continue
                
            # Read the segmentation mask
            mask_data = imageio.imread(seg_file_path)
            
            # Store in zarr format for efficient processing
            zarr_file_path = store_annotations_in_zarr(mask_data, zarr_path, global_n)
            
            # Store embedding in zarr format and zip for OMERO upload
            embed_zarr = f"embedding_{local_n:05d}.zarr"
            embed_dir = os.path.join(output_folder, "embed")
            zip_path = os.path.join(embed_dir, f"embedding_{global_n:05d}.zip")
            
            with zipfile.ZipFile(zip_path, 'w') as zip_file:
                zip_directory(embed_dir, embed_zarr, zip_file)
            
            # Upload embedding to OMERO
            embed_id = ezomero.post_file_annotation(
                conn,
                str(zip_path),
                ns='microsam.embeddings',
                object_type="Image",
                object_id=image.getId(),
                description=f'SAM embedding ({model_type})'
            )
            
            # Convert zarr annotation to TIFF for OMERO compatibility
            tiff_path = os.path.join(output_folder, "output", f"seg_{global_n:05d}.tiff")
            zarr_to_tiff(zarr_file_path, tiff_path)
            
            # Upload labels and create ROIs
            label_id, roi_id = upload_rois_and_labels(
                conn, 
                image, 
                tiff_path, 
                z_slice, 
                channel, 
                timepoint, 
                model_type
            )
            
            # Update tracking dataframe
            is_train = combined_images_sequence[global_n] == 0 if not segment_all else True
            is_validate = combined_images_sequence[global_n] == 1 if not segment_all else False
            
            new_row = pd.DataFrame([{
                "image_id": image.getId(),
                "image_name": image.getName(),
                "train": is_train,
                "validate": is_validate,
                "channel": channel,
                "z_slice": z_slice,
                "timepoint": timepoint,
                "sam_model": model_type,
                "embed_id": embed_id,
                "label_id": label_id,
                "roi_id": roi_id
            }])
            df = pd.concat([df, new_row], ignore_index=True)
            
        # Clean up temporary files for this batch
        for n in range(batch_size):  # Use local indexing for cleanup
            if start_idx + n >= len(combined_images):  # Skip if we've processed all images
                continue
                
            embed_zip = os.path.join(output_folder, "embed", f"embedding_{n:05d}.zip")
            embed_zarr = os.path.join(output_folder, "embed", f"embedding_{n:05d}.zarr")
            seg_file = os.path.join(output_folder, "output", f"seg_{n:05d}.tif")
            
            for path in [embed_zip, seg_file]:
                if os.path.exists(path):
                    os.remove(path)
                    
            if os.path.exists(embed_zarr) and os.path.isdir(embed_zarr):
                shutil.rmtree(embed_zarr)
    
    # Upload final tracking table to OMERO
    table_id = ezomero.post_table(
        conn, 
        object_type="Dataset", 
        object_id=dataset.getId(), 
        table=df,
        title="micro_sam_training_data"
    )
    
    print(f"All processing complete. Uploaded tracking table with ID: {table_id}")
    return table_id, combined_images

### Demo: Running segmentation with Dask lazy loading

This cell demonstrates using dask for lazy loading from OMERO and zarr for annotation storage.

In [ ]:
# Set parameters for dask-based batch processing
model_type = 'vit_b'
segment_all = False
train_n = 2   
validate_n = 2
channel = 3
timepoint = 0
z_slice = 4
batch_size = 2

# Configure napari settings
settings = get_settings()
settings.application.ipy_interactive = False

# Run batch processing with dask lazy loading
if datatype == "dataset":
    table_id, processed_images = process_omero_batch_with_dask(
        dataset=dataset,
        output_folder=output_directory,
        model_type=model_type,
        batch_size=batch_size,
        channel=channel,
        timepoint=timepoint,
        z_slice=z_slice,
        segment_all=segment_all,
        train_n=train_n,
        validate_n=validate_n,
    )
    print(f"Finished processing with dask lazy loading. Table ID: {table_id}")